In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup    # for web scraping
import urllib.request

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = urllib.request.urlopen(url)
article = req.read().decode()

with open("List_of_postal_codes_of_Canada:_M", 'w') as fo:
    fo.write(article)

In [3]:
# Load article, turn into soup and get the <table>s.
article = open("List_of_postal_codes_of_Canada:_M").read()
soup = BeautifulSoup(article, 'html.parser')
tables = soup.find_all('table', class_='sortable')

In [4]:
# Search through the tables for the one with the headings we want.
for table in tables:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    
# Obtain the information we want from each row
for table in tables:
    row = table.find_all('td')
    cot = [td.text.strip() for td in row]

In [5]:
# create the dataframe that obtains all the row
toronto = pd.DataFrame(cot)

# reshape the dateframe and rename the columns by headings
toronto = pd.DataFrame(toronto.values.reshape(180, 3), columns=headings)

In [6]:
toronto.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [7]:
# find the rows that have Not Assigned in Borough
toronto.loc[toronto["Borough"] == "Not assigned"]

# Drop these rows
toronto.drop(toronto.loc[toronto["Borough"] == "Not assigned"].index, inplace=True)

In [8]:
toronto

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [9]:
# check if there is any not assigned neighborhood
toronto.loc[toronto["Neighborhood"] == "Not assigned"]

,Postal Code,Borough,Neighborhood


In [10]:
# Check the shape of the Dataframe
toronto.shape

(103, 3)

In [22]:
# load the geographical coordinates of each postal code
geo = pd.read_csv("Geospatial_Coordinates.csv")

# take a look of the shape of the postal code dataframe
geo.shape

(103, 3)

Match with the dataframe that we scrape from the wiki page

In [25]:
# Do a outer join to join the 2 dataframes and save it in a variable
toronto_geo = pd.merge(toronto, geo, on="Postal Code", how="outer")

In [ ]:
toronto